In [71]:
import numpy as np
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import plotly.express as px
from Connections import Connections


In [3]:
bd = Connections()

In [4]:
df = bd.get_orders_data_linear_regression()

In [5]:
df

,order_created_at_hour,day_of_month,channel_id,is_weekend,day_of_week,order_hour,order_month,channel_type,channel_type2,hub_city_curitiba,hub_city_rj,hub_city_pa,hub_state_rs,hub_state_sp,hub_state_rj,order_count
0,2021-02-01 00:00:00,1,5,0,1.0,0,2,1,MARKETPLACE,0,0,0,0,1,0,8
1,2021-02-01 00:00:00,1,15,0,1.0,0,2,1,MARKETPLACE,0,0,0,0,1,0,1
2,2021-03-01 00:00:00,1,5,0,1.0,0,3,1,MARKETPLACE,0,0,0,0,1,0,6
3,2021-04-01 00:00:00,1,1,0,4.0,0,4,2,OWN CHANNEL,0,0,0,0,1,0,1
4,2021-04-01 00:00:00,1,5,0,4.0,0,4,1,MARKETPLACE,0,0,0,0,1,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,2021-01-31 23:00:00,31,1,1,0.0,23,1,2,OWN CHANNEL,0,0,0,0,1,0,1
2524,2021-01-31 23:00:00,31,5,1,0.0,23,1,1,MARKETPLACE,0,0,0,0,1,0,14
2525,2021-01-31 23:00:00,31,31,1,0.0,23,1,1,MARKETPLACE,0,0,0,0,1,0,1
2526,2021-03-31 23:00:00,31,1,0,3.0,23,3,2,OWN CHANNEL,0,0,0,0,1,0,1


In [6]:
# Cria Array com X e Y sendo X as variaveis que tentam explicar as quantidades de pedidos no array Y 
X = []
Y = []
for x in range(len(df)):
    aux = []
    aux = [
                df["is_weekend"][x],
                float(df["day_of_month"][x]),   
                df["day_of_week"][x],
                df["hub_state_rs"][x] ,
                df["hub_state_sp"][x] ,
                df["hub_state_rj"][x] ,
                df["hub_city_curitiba"][x],
                df["hub_city_rj"][x],
                df["hub_city_pa"][x],
                df["channel_type"][x],
                df["channel_id"][x],
                df["order_hour"][x],
                df["order_month"][x],
                
          ]
    X.append(aux)
    Y.append(df["order_count"][x])

x, y = np.array(X), np.array(Y)


In [7]:
#Ajustando dados no padrão utilizado pelo statsmodels  
x = sm.add_constant(x)

model = sm.OLS(y, x)


results = model.fit()


results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.344
Method:                 Least Squares   F-statistic:                     190.7
Date:                Tue, 27 Sep 2022   Prob (F-statistic):          2.82e-227
Time:                        21:12:07   Log-Likelihood:                -7458.6
No. Observations:                2528   AIC:                         1.493e+04
Df Residuals:                    2520   BIC:                         1.498e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1.6085      0.198      8.120      0.000       1.220       1.997
x2             0.0514      0.011      4.759      0.000       0.030       0.073
x3            -0.0364      0.049     -0.743      0.458      -0.132       0.060
const       1.526e-14   6.26e-16     24.365      0.000     1.4e-14    1.65e-14
x4            12.0275      0.509     23.644      0.000      11.030      13.025
x5         -1.221e-15   5.92e-17    -20.631      0.000   -1.34e-15    -1.1e-15
x6         -4.045e-17   3.96e-18    -10.223      0.000   -4.82e-17   -3.27e-17
x7          1.925e-15   6.13e-17     31.390      0.000    1.81e-15    2.05e-15
x8                  0          0        nan        nan           0           0
x9            -6.9284      0.208    -33.337      0.000      -7.336      -6.521
x10           -0.3606      0.018    -19.976      0.000      -0.396      -0.325
x11            0.0796      0.015      5.281      0.000       0.050       0.109
x12            0.7728      0.083      9.280      0.000       0.609       0.936
==============================================================================
Omnibus:                      660.875   Durbin-Watson:                   1.911
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1914.428
Skew:                           1.346   Prob(JB):                         0.00
Kurtosis:                       6.306   Cond. No.                     9.31e+33
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.84e-62. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [8]:
print('coefficient of determination:', results.rsquared)
print('adjusted coefficient of determination:', results.rsquared_adj)
print('regression coefficients:', results.params)

coefficient of determination: 0.3463085663800176
adjusted coefficient of determination: 0.3444927568421844
regression coefficients: [ 1.60845691e+00  5.13910279e-02 -3.63919280e-02  1.52611833e-14
  1.20274544e+01 -1.22088025e-15 -4.04493218e-17  1.92533303e-15
  0.00000000e+00 -6.92841283e+00 -3.60579214e-01  7.96287126e-02
  7.72753945e-01]


In [9]:
#PREDIÇAO
print('fitted response:', results.fittedvalues, sep='\n')
print('predicted response:', results.predict(x), sep='\n')

fitted response:
[ 4.85665245  1.25086031  5.62940639 ... -0.27312099  3.44371779
  8.92981376]
predicted response:
[ 4.85665245  1.25086031  5.62940639 ... -0.27312099  3.44371779
  8.92981376]


Apresentação dos dados Graficamente

In [67]:
# df["order_created_at_hour"] = pd.to_datetime(df["order_created_at_hour"])
# df.set_index('order_created_at_hour', inplace=True)
# # len(results.predict())
df_predict = pd.DataFrame(results.predict())
df_predict.index = df.index

df_predict.rename(columns={0: 'order_predict'}, inplace=True)

concat_df = pd.concat([df,df_predict["order_predict"]], axis=1)


# concat_df.set_index(concat_df.order_created_at_hour, inplace=True)

In [69]:
fig = px.line(concat_df, x=concat_df.index, y=["order_count","order_predict"], template = "plotly_dark")
fig.show()

NameError: name 'px' is not defined